In [1]:
from pyflink.table import (
    EnvironmentSettings,
    TableEnvironment,
    TableDescriptor,
    Schema,
    DataTypes,
    FormatDescriptor,
    EnvironmentSettings
)
from pyflink.table.window import Tumble,Slide, Over
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import expressions as expr
from pyflink.table.expressions import col,call, lit
from pyflink.table.udf import udf

In [2]:
env_settings = EnvironmentSettings.in_streaming_mode()
t_env = TableEnvironment.create(env_settings)    # Point to the SQL Kafka connector JAR
kafka_connector_jar = "file:///Users/gerardomartinez/DataEng/alpaca/flink-connectors/flink-sql-connector-kafka-3.3.0.jar"

t_env.get_config().set("pipeline.jars", kafka_connector_jar)

dockerC = "localhost:9092"

# t_env.get_config().set("table.exec.emit.early-fire.enabled", "false")
# t_env.get_config().set("table.exec.emit.allow-lateness", "0s")

In [3]:
source_ddl = f"""
        create table fake_stocks(
            stock_name STRING,
            stock_value DOUBLE,
            log_time AS PROCTIME()
        ) with (
            'connector' = 'kafka',
            'topic' = 'aapl_price',
            'properties.bootstrap.servers' = '{dockerC}',
            'properties.group.id' = 'transaction_group',
            'scan.startup.mode' = 'earliest-offset',
            'format' = 'json',
            'json.fail-on-missing-field' = 'false',
            'json.ignore-parse-errors' = 'true'
        );
        """

t_env.execute_sql(source_ddl)

In [4]:
query = """
WITH current_and_previous as (
    select
        stock_name,
        log_time,
        stock_value,
        lag(stock_value, 1) over (partition by stock_name order by log_time) previous_value
    from fake_stocks
)
select *,
    case
        when stock_value > previous_value then '▲'
        when stock_value < previous_value then '▼'
        else '='
    end as trend
FROM TABLE(
    TUMBLE(TABLE current_and_previous, DESCRIPTOR(log_time), INTERVAL '1' MINUTE)
);
"""

t_results = t_env.execute_sql(query)

In [5]:
t_results.print()

+----+--------------------------------+-------------------------+--------------------------------+--------------------------------+-------------------------+-------------------------+-------------------------+--------------------------------+
| op |                     stock_name |                log_time |                    stock_value |                 previous_value |            window_start |              window_end |             window_time |                          trend |
+----+--------------------------------+-------------------------+--------------------------------+--------------------------------+-------------------------+-------------------------+-------------------------+--------------------------------+
| +I |                           AAPL | 2025-02-13 11:11:15.591 |                         240.93 |                         <NULL> | 2025-02-13 11:11:00.000 | 2025-02-13 11:12:00.000 | 2025-02-13 11:11:15.591 |                              = |
| +I |                      

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/gerardomartinez/DataEng/.venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/gerardomartinez/DataEng/.venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/gerardomartinez/.pyenv/versions/3.10.4/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

| +I |                           AAPL | 2025-02-13 17:35:38.981 |                         241.48 |                         241.48 | 2025-02-13 17:35:00.000 | 2025-02-13 17:36:00.000 | 2025-02-13 17:35:38.981 |                              = |
| +I |                           AAPL | 2025-02-13 17:36:09.073 |                         241.48 |                         241.48 | 2025-02-13 17:36:00.000 | 2025-02-13 17:37:00.000 | 2025-02-13 17:36:09.073 |                              = |
